# Group 17: 
### Eyad Medhat 221100279 / Hady Aly 221101190 / Mohamed Mahfouz 221101743 / Omar Mady 221100745

# PCA with Mean Filling
Eyad Medhat 221100279/ Hady Aly 221101190 / Mohamed Mahfouz 221101743 / Omar Mady 221100745

Task 1: Calculate Average Rating for Target Items I1 and I2

In [1]:
from utils import *

Results folder exists at: d:\University\semester 9\IRS\AIE425_FinalProject_Group17\SECTION1_DimensionalityReduction\results
Subfolder exists: d:\University\semester 9\IRS\AIE425_FinalProject_Group17\SECTION1_DimensionalityReduction\results\plots
Subfolder exists: d:\University\semester 9\IRS\AIE425_FinalProject_Group17\SECTION1_DimensionalityReduction\results\tables


In [2]:
# 1. Load the Target Items (I1, I2) from the previous results
target_items_df = load_data(table_name='lowest_two_rateditems.csv')

if target_items_df is not None:
    print("Target Items Loaded:")
    print(target_items_df)
    
    # Access by position to ensure we get the first two rows regardless of index
    I1 = target_items_df.iloc[0]['movieId']
    I2 = target_items_df.iloc[1]['movieId']
    print(f"Target Item 1 (I1): {I1}")
    print(f"Target Item 2 (I2): {I2}")
else:
    print("ERROR: Target items file not found. Make sure previous steps were run.")

 Found requested table at: ..\results\tables\lowest_two_rateditems.csv
Target Items Loaded:
   movieId  mean_rating_per_movie  rating_count_per_movie
0     1556               1.919431                     422
1     1499               2.059603                     453
Target Item 1 (I1): 1556.0
Target Item 2 (I2): 1499.0


In [3]:
# 2. Load the Ratings Data
ratings_df = load_data()

if ratings_df is not None:
    print(f"Ratings loaded. Shape: {ratings_df.shape}")
else:
    print("FAILED to load ratings data.")

 Found cached sample at: ..\data\ml-20m\ratings_cleaned_sampled.csv
Ratings loaded. Shape: (1000000, 3)


In [4]:
# 3. Calculate Average Rating for I1 and I2
if ratings_df is not None and 'I1' in locals():
    # Filter for I1
    i1_ratings = ratings_df[ratings_df['movieId'] == I1]
    i1_mean = i1_ratings['rating'].mean()
    i1_count = len(i1_ratings)
    
    # Filter for I2
    i2_ratings = ratings_df[ratings_df['movieId'] == I2]
    i2_mean = i2_ratings['rating'].mean()
    i2_count = len(i2_ratings)
    
    print(f"\n--- Calculated Stats ---")
    print(f"Item I1 (ID: {I1}): Mean Rating = {i1_mean:.4f}, Count = {i1_count}")
    print(f"Item I2 (ID: {I2}): Mean Rating = {i2_mean:.4f}, Count = {i2_count}")
    
    # Verification against loaded values
    print(f"\n--- Verification ---")
    print(f"Stored I1 Mean: {target_items_df.iloc[0]['mean_rating_per_movie']}")
    print(f"Stored I2 Mean: {target_items_df.iloc[1]['mean_rating_per_movie']}")
    
    # --- SAVE RESULT ---
    task1_data = [
        {'movieId': I1, 'mean_rating': i1_mean, 'count': i1_count},
        {'movieId': I2, 'mean_rating': i2_mean, 'count': i2_count}
    ]
    task1_df = pd.DataFrame(task1_data)
    save_csv(task1_df, 'task3.2.1.csv')
    
else:
    print("Cannot calculate stats: Missing data.")


--- Calculated Stats ---
Item I1 (ID: 1556.0): Mean Rating = 1.9194, Count = 422
Item I2 (ID: 1499.0): Mean Rating = 2.0596, Count = 453

--- Verification ---
Stored I1 Mean: 1.919431279620853
Stored I2 Mean: 2.0596026490066226
    Saved CSV: tables/task3.2.1.csv


## Task 2: Mean Filling and Dataset Augmentation
Fill missing ratings for Target Items I1 and I2 with their mean value (1.0) and save the augmented dataset.

In [5]:
augmented_df = None # Initialize
if ratings_df is not None and 'I1' in locals():
    print("Augmenting dataset with filled ratings for I1 and I2...")
    
    all_users = ratings_df['userId'].unique()
    print(f"Total unique users: {len(all_users)}")
    
    new_rows = []
    
    # --- Process I1 ---
    # Find users who rated I1
    users_rated_i1 = set(ratings_df[ratings_df['movieId'] == I1]['userId'].unique())
    # Find missing
    users_missing_i1 = [u for u in all_users if u not in users_rated_i1]
    print(f"Users missing rating for I1: {len(users_missing_i1)}")
    
    # Create rows for missing I1
    for u in users_missing_i1:
        new_rows.append({'userId': u, 'movieId': I1, 'rating': i1_mean})
        
    # --- Process I2 ---
    # Find users who rated I2
    users_rated_i2 = set(ratings_df[ratings_df['movieId'] == I2]['userId'].unique())
    # Find missing
    users_missing_i2 = [u for u in all_users if u not in users_rated_i2]
    print(f"Users missing rating for I2: {len(users_missing_i2)}")
    
    # Create rows for missing I2
    for u in users_missing_i2:
        new_rows.append({'userId': u, 'movieId': I2, 'rating': i2_mean})
    
    print(f"Creating dataframe for {len(new_rows)} new ratings...")
    new_ratings_df = pd.DataFrame(new_rows)
    
    # Concatenate
    print("Appending to original dataset...")
    augmented_df = pd.concat([ratings_df, new_ratings_df], ignore_index=True)
    
    # Sort for tidiness (optional)
    augmented_df.sort_values(by=['userId', 'movieId'], inplace=True)
    
    print(f"Original Shape: {ratings_df.shape}")
    print(f"Augmented Shape: {augmented_df.shape}")
    
    # Verify counts
    final_i1_count = len(augmented_df[augmented_df['movieId'] == I1])
    final_i2_count = len(augmented_df[augmented_df['movieId'] == I2])
    print(f"Final Count I1: {final_i1_count} (Should be {len(all_users)}) ")
    print(f"Final Count I2: {final_i2_count} (Should be {len(all_users)}) ")
    
    # --- SAVE RESULT ---
    save_csv(augmented_df, 'ratings_filled_targets.csv')
    print("Save Complete.")
    
else:
    print("Skipping augmentation due to missing data.")

Augmenting dataset with filled ratings for I1 and I2...
Total unique users: 96345
Users missing rating for I1: 95923
Users missing rating for I2: 95892
Creating dataframe for 191815 new ratings...
Appending to original dataset...
Original Shape: (1000000, 3)
Augmented Shape: (1191815, 3)
Final Count I1: 96345 (Should be 96345) 
Final Count I2: 96345 (Should be 96345) 
    Saved CSV: tables/ratings_filled_targets.csv
Save Complete.


## Task 3: Calculate Average Ratings from Augmented Data
Calculate the mean rating for each item in the new augmented dataset.

In [6]:
final_stats = None # Initialize
if augmented_df is not None:
    print("Calculating stats for augmented dataset...")
    
    # Calculate stats
    final_stats = augmented_df.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
    final_stats.rename(columns={'mean': 'mean_rating', 'count': 'rating_count'}, inplace=True)
    
    print("Stats calculated.")
    print(final_stats.head())
    
    # Verification for I1 and I2
    if 'I1' in locals():
        i1_final = final_stats[final_stats['movieId'] == I1]
        print(f"\nFinal Stats I1:\n{i1_final}")
        
        i2_final = final_stats[final_stats['movieId'] == I2]
        print(f"Final Stats I2:\n{i2_final}")
        
    # --- SAVE RESULT ---
    print("Saving final item stats...")
    save_csv(final_stats, 'task3.2.3_final_item_stats.csv')
    print("Task 3 Complete.")
    
else:
    print("Augmented dataframe not available.")

Calculating stats for augmented dataset...
Stats calculated.
   movieId  mean_rating  rating_count
0      1.0     3.915355          3875
1      2.0     3.202874          1740
2      3.0     3.209073          1014
3      5.0     3.061538           975
4      6.0     3.770856          1846

Final Stats I1:
     movieId  mean_rating  rating_count
429   1556.0     1.919431         96345
Final Stats I2:
     movieId  mean_rating  rating_count
422   1499.0     2.059603         96345
Saving final item stats...
    Saved CSV: tables/task3.2.3_final_item_stats.csv
Task 3 Complete.


## Task 4: Calculate Diffrence between Ratings and Mean Rating
Load the augmented ratings and final item stats from CSVs, then calculate the difference.

In [7]:
print("Loading files for Task 4...")

task4_ratings = load_data(table_name='ratings_filled_targets.csv')
task4_stats = load_data(table_name='task3.2.3_final_item_stats.csv')

if task4_ratings is not None and task4_stats is not None:
    print(f"Loaded Augmented Ratings. Shape: {task4_ratings.shape}")
    print(f"Loaded Item Stats. Shape: {task4_stats.shape}")
    
    print("Calculating rating differences (Centering)...")
    
    # Merge
    merged_df = task4_ratings.merge(task4_stats[['movieId', 'mean_rating']], on='movieId', how='left')
    
    # Calculate difference
    merged_df['rating_diff'] = merged_df['rating'] - merged_df['mean_rating']
    
    print("Difference calculated.")
    print(merged_df[['userId', 'movieId', 'rating', 'mean_rating', 'rating_diff']].head())
    
    # Verify centering (Mean difference should be close to 0)
    avg_diff = merged_df['rating_diff'].mean()
    print(f"\nAverage Global Difference (should be ~0): {avg_diff:.6f}")
    
    # --- SAVE RESULT ---
    print("Saving centered ratings...")
    save_csv(merged_df, 'task3.2.4_centered_ratings.csv')
    print("Task 4 Complete.")
    
else:
    print("Cannot proceed with Task 4 due to missing files.")

Loading files for Task 4...
 Found requested table at: ..\results\tables\ratings_filled_targets.csv
 Found requested table at: ..\results\tables\task3.2.3_final_item_stats.csv
Loaded Augmented Ratings. Shape: (1191815, 3)
Loaded Item Stats. Shape: (1000, 3)
Calculating rating differences (Centering)...
Difference calculated.
   userId  movieId  rating  mean_rating  rating_diff
0       1     32.0     4.0     3.881264     0.118736
1       1    337.0     4.0     3.755682     0.244318
2       1   1193.0     4.0     4.191716    -0.191716
3       1   1261.0     4.0     3.848649     0.151351
4       1   1370.0     3.0     3.454844    -0.454844

Average Global Difference (should be ~0): -0.000000
Saving centered ratings...
    Saved CSV: tables/task3.2.4_centered_ratings.csv
Task 4 Complete.


## Task 5: Calculate Partial Covariance (Targets vs All)
Compute the covariance where we calculate Cov(Target, Item_j) for all Items j.
We use the helper function from utils to do this efficiently.

In [8]:
target_items_df = load_data(table_name='lowest_two_rateditems.csv')

if target_items_df is not None:
    I1 = target_items_df.iloc[0]['movieId']
    I2 = target_items_df.iloc[1]['movieId']
    print(f" Target Items IDs found: {I1}, {I2}")

    # 2. Load Centered Ratings
    # Note: load_data loads full CSV. 
    df = load_data(table_name='task3.2.4_centered_ratings.csv')
    
    if df is not None:
        # 3. Calculate Target Covariances using Utils
        print("Calculating partial covariance matrix (2 x All Items)...")
        
        targets = [I1, I2]
        partial_cov_df = calculate_target_covariance(df, targets)
        
        print("Calculation Complete.")
        print(f"Result Shape: {partial_cov_df.shape}")
        print("Sample (first 5 cols):")
        print(partial_cov_df.iloc[:, :5])
        
        # Task 5 Save
        save_csv(partial_cov_df, '3.2.5_target_only_covariances.csv')
        print("Task 5 Complete.")
    else:
        print("Centered ratings data missing.")
else:
    print("Target items data missing.")

 Found requested table at: ..\results\tables\lowest_two_rateditems.csv
 Target Items IDs found: 1556.0, 1499.0
 Found requested table at: ..\results\tables\task3.2.4_centered_ratings.csv
Calculating partial covariance matrix (2 x All Items)...
Starting MANUAL covariance calculation...
Total N (Users): 96345
Total Items: 1000
Users who rated targets: 96345
Building efficient lookup dictionary...
Lookup built. Calculating sums...
Processing Target Item: 1556.0...
Processing Target Item: 1499.0...
Formatting results...
Calculation Complete.
Result Shape: (2, 1000)
Sample (first 5 cols):
             1.0      2.0       3.0       5.0       6.0
1556.0  0.000013  0.00007 -0.000016  0.000042  0.000027
1499.0  0.000058  0.00010  0.000024  0.000046  0.000014
    Saved CSV: tables/3.2.5_target_only_covariances.csv
Task 5 Complete.


## Task 6: Reference Full Covariance Matrix Calculation
Calculates the FULL N x N covariance matrix for all items using sparse algebra and saves it as a compressed .npz file.

In [9]:
print("Loading centered ratings for Full Matrix calculation...")
df_full = load_data(table_name='task3.2.4_centered_ratings.csv')

if df_full is not None:
    print("Calculating Full Sparse Matrix (N x N)...")
    sparse_cov, movie_ids = calculate_full_covariance_sparse(df_full)
    
    print(f"Matrix Shape: {sparse_cov.shape}")
    
    # Save Matrix (Sparse) - using global results_root from utils for robust path
    out_path = os.path.join(results_root, 'tables', '3.2.6_full_covariance.npz')
    print(f"Saving Sparse Matrix to {out_path}...")
    sparse.save_npz(out_path, sparse_cov)
    
    # Save ID Mapping
    pd.DataFrame(movie_ids, columns=['movieId'])
    save_csv(pd.DataFrame(movie_ids, columns=['movieId']), '3.2.6_full_covariance_ids.csv')
    
    # --- MASSIVE CSV WRITE WITH CHUNKING (PARTIAL 60%) ---
    # Since save_csv simplifies overwriting, we handle chunking manually but use path from utils if possible
    # or just use os.path.join with results_root
    full_csv_path = os.path.join(results_root, 'tables', '3.2.6_full_covariance_partial.csv')
    print(f"Writing PARTIAL (60%) Full Matrix to CSV: {full_csv_path}")
    print("Writing in chunks...")
    
    chunk_size = 1000
    num_items = sparse_cov.shape[0]
    limit_items = int(num_items * 0.60) # 60% Limit
    
    print(f"Total items: {num_items}. Writing first {limit_items} items.")
    
    start_time = time.time()
    
    for i in range(0, limit_items, chunk_size):
        end_i = min(i + chunk_size, limit_items)
        
        # Extract chunk and make dense
        chunk_dense = sparse_cov[i:end_i].toarray()
        
        # Create DF
        chunk_index = movie_ids[i:end_i]
        chunk_df = pd.DataFrame(chunk_dense, index=chunk_index, columns=movie_ids)
        
        # Write mode: 'w' for first chunk, 'a' for others
        if i == 0:
            chunk_df.to_csv(full_csv_path, mode='w', header=True)
        else:
            chunk_df.to_csv(full_csv_path, mode='a', header=False)
            
        # Progress log
        if i % 3000 == 0:
            elapsed = time.time() - start_time
            print(f"  Written rows {i} to {end_i} / {limit_items} ({elapsed:.1f}s)")
    
    print(f"CSV Write Complete. Time: {time.time() - start_time:.1f}s")
    print("Task 6 Complete.")
    
else:
    print("Input file not found.")

Loading centered ratings for Full Matrix calculation...
 Found requested table at: ..\results\tables\task3.2.4_centered_ratings.csv
Calculating Full Sparse Matrix (N x N)...
Preparing for Full Sparse Covariance Calculation...
Dimensions: 96345 Users x 1000 Items
Constructing User-Item Sparse Matrix...
Computing X.T @ X ... (This may take a moment)
Dividing by N-1...
Matrix Shape: (1000, 1000)
Saving Sparse Matrix to d:\University\semester 9\IRS\AIE425_FinalProject_Group17\SECTION1_DimensionalityReduction\results\tables\3.2.6_full_covariance.npz...
    Saved CSV: tables/3.2.6_full_covariance_ids.csv
Writing PARTIAL (60%) Full Matrix to CSV: d:\University\semester 9\IRS\AIE425_FinalProject_Group17\SECTION1_DimensionalityReduction\results\tables\3.2.6_full_covariance_partial.csv
Writing in chunks...
Total items: 1000. Writing first 600 items.
  Written rows 0 to 600 / 600 (0.6s)
CSV Write Complete. Time: 0.6s
Task 6 Complete.


## Task 7: Determine Top 5 and Top 10 Peers
Identify the items with the highest covariance with each target item (I1 and I2) using the generated full matrix.

In [10]:
# Task 7 - Identify Item Peers Using PCA Latent Space
import pandas as pd
import numpy as np
import os
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

matrix_path = os.path.join('..', 'results', 'tables', '3.2.6_full_covariance.npz')
id_path = os.path.join('..', 'results', 'tables', '3.2.6_full_covariance_ids.csv')
target_items_path = os.path.join('..', 'results', 'tables', 'lowest_two_rateditems.csv')

if os.path.exists(matrix_path) and os.path.exists(id_path) and os.path.exists(target_items_path):
    print("Loading Full Sparse Matrix...")
    cov_matrix_sparse = sparse.load_npz(matrix_path)
    cov_matrix = cov_matrix_sparse.toarray() # PCA needs dense matrix usually
    
    print("Loading IDs...")
    movie_ids_df = pd.read_csv(id_path)
    all_movie_ids = movie_ids_df['movieId'].tolist()
    
    print("Loading Targets...")
    targets_df = pd.read_csv(target_items_path)
    targets = [targets_df.iloc[0]['movieId'], targets_df.iloc[1]['movieId']]
    
    # --- PCA Eigen-decomposition ---
    print("Performing Eigen-decomposition...")
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    
    # Sort descending
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]
    
    # Determine k for 75% Variance
    total_variance = np.sum(eigenvalues)
    explained_variance_ratio = eigenvalues / total_variance
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
    k_75 = np.argmax(cumulative_variance >= 0.75) + 1
    print(f"Number of components to explain 75% variance: {k_75}")
    print(f"Correlation Variance at k={k_75}: {cumulative_variance[k_75-1]:.4f}")
    
    # Construct Latent Item Features
    latent_item_features = eigenvectors[:, :k_75]
    latent_item_df = pd.DataFrame(latent_item_features, index=all_movie_ids)
    
    results = []
    
    for tid in targets:
        print(f"\nProcessing Peers for Target: {tid} (Latent Space)")
        if tid in latent_item_df.index:
            target_vec = latent_item_df.loc[tid].values.reshape(1, -1)
            
            # Compute Cosine Similarity
            sim_scores = cosine_similarity(target_vec, latent_item_df.values).flatten()
            sim_series = pd.Series(sim_scores, index=latent_item_df.index)
            
            # Exclude self
            sim_series = sim_series.drop(tid)
            
            # Sort & Top 10
            top_10 = sim_series.sort_values(ascending=False).head(10)
            
            print(f"Top 5 Peers for {tid}:")
            print(top_10.head(5).index.tolist())

            print(f"Top 10 Peers for {tid}:")
            print(top_10.index.tolist())
            
            # Store results
            rank = 1
            for peer_id, sim in top_10.items():
                results.append({
                    'Target_Item': tid,
                    'Rank': rank,
                    'Peer_Item': peer_id,
                    'Latent_Similarity': sim # Storing similarity weight
                })
                rank += 1
        else:
            print(f"Target {tid} not found in matrix indices.")
            
    # Save Results
    results_df = pd.DataFrame(results)
    out_path = os.path.join('..', 'results', 'tables', '3.2.7_top_peers.csv')
    
    print(f"\nSaving Top Peers to {out_path}...")
    results_df.to_csv(out_path, index=False)
    print("Task 7 Complete.")
    
else:
    print("Missing required files for Task 7. Run Task 6 first.")

Loading Full Sparse Matrix...
Loading IDs...
Loading Targets...
Performing Eigen-decomposition...
Number of components to explain 75% variance: 504
Correlation Variance at k=504: 0.7503

Processing Peers for Target: 1556.0 (Latent Space)
Top 5 Peers for 1556.0:
[2421.0, 1722.0, 256.0, 1831.0, 1370.0]
Top 10 Peers for 1556.0:
[2421.0, 1722.0, 256.0, 1831.0, 1370.0, 2431.0, 2407.0, 2134.0, 1591.0, 79.0]

Processing Peers for Target: 1499.0 (Latent Space)
Top 5 Peers for 1499.0:
[2722.0, 1347.0, 1591.0, 3826.0, 1438.0]
Top 10 Peers for 1499.0:
[2722.0, 1347.0, 1591.0, 3826.0, 1438.0, 2949.0, 1042.0, 2067.0, 799.0, 379.0]

Saving Top Peers to ..\results\tables\3.2.7_top_peers.csv...
Task 7 Complete.


## Task 8: Determine Reduced Dimensional Space (Top 5 Peers)
Create a reduced dimensional space for each user by selecting their centered ratings for the Top 5 peer items identified in Task 7.
Missing values are filled with 0 (since ratings are centered).

In [11]:
# Task 8: Reduced Dimensional Space (Save Weights & User Vectors)
import pandas as pd
import os
import utils

peers_path = os.path.join('..', 'results', 'tables', '3.2.7_top_peers.csv')
centered_path = os.path.join('..', 'results', 'tables', 'task3.2.4_centered_ratings.csv')

if os.path.exists(peers_path) and os.path.exists(centered_path):
    print("Loading Top Peers and Centered Ratings...")
    peers_df = pd.read_csv(peers_path)
    centered_df = pd.read_csv(centered_path, usecols=['userId', 'movieId', 'rating_diff'])
    
    all_users = centered_df['userId'].unique()
    print(f"Total Users: {len(all_users)}")
    
    targets = peers_df['Target_Item'].unique()
    
    for target in targets:
        print(f"\n--- Processing Target {target} ---")
        
        # 1. Identify Top 5 Peers
        target_peers = peers_df[peers_df['Target_Item'] == target].sort_values('Rank')
        top_5_peers = target_peers.head(5)['Peer_Item'].tolist()
        print(f"Top 5 Peers: {top_5_peers}")
        
        # 2. Filter Centered Ratings
        subset_df = centered_df[centered_df['movieId'].isin(top_5_peers)]
        
        # 3. Create User Vectors (For verification/pipeline consistency)
        # We pivot to get the user's ratings for the top 5 peers
        print("Creating Reduced Space Matrix (User Vectors)...")
        reduced_matrix = subset_df.pivot(index='userId', columns='movieId', values='rating_diff')
        reduced_matrix = reduced_matrix.fillna(0.0)
        reduced_matrix = reduced_matrix.reindex(all_users, fill_value=0.0)
        
        print(f"Reduced Matrix Shape: {reduced_matrix.shape}")
        
        # 4. Save User Vectors
        out_filename = f'3.2.8_reduced_space_target_{int(target)}.csv'
        print(f"Saving User Vectors to {out_filename}...")
        utils.save_csv(reduced_matrix.reset_index(), out_filename)
    
    print("\nTask 8 Complete.")
else:
    print("Error: Required files not found.")

Loading Top Peers and Centered Ratings...
Total Users: 96345

--- Processing Target 1556.0 ---
Top 5 Peers: [2421.0, 1722.0, 256.0, 1831.0, 1370.0]
Creating Reduced Space Matrix (User Vectors)...
Reduced Matrix Shape: (96345, 5)
Saving User Vectors to 3.2.8_reduced_space_target_1556.csv...
    Saved CSV: tables/3.2.8_reduced_space_target_1556.csv

--- Processing Target 1499.0 ---
Top 5 Peers: [2722.0, 1347.0, 1591.0, 3826.0, 1438.0]
Creating Reduced Space Matrix (User Vectors)...
Reduced Matrix Shape: (96345, 5)
Saving User Vectors to 3.2.8_reduced_space_target_1499.csv...
    Saved CSV: tables/3.2.8_reduced_space_target_1499.csv

Task 8 Complete.


## Task 9: Rating Prediction using Linear Regression (Top 5 Peers)
Use the reduced dimensional space (Top 5 Peers) to train a Linear Regression model for each target item.
Predict the centered ratings for users who haven't rated the target item, then add the mean rating to get the final prediction.

In [12]:
# Task 9: Prediction using Weighted Average (Top 5 Peers)
import pandas as pd
import os
import utils

stats_path = os.path.join('..', 'results', 'tables', 'task3.2.3_final_item_stats.csv')
centered_path = os.path.join('..', 'results', 'tables', 'task3.2.4_centered_ratings.csv')
peers_path = os.path.join('..', 'results', 'tables', '3.2.7_top_peers.csv') # Contains Weights

if os.path.exists(stats_path) and os.path.exists(centered_path) and os.path.exists(peers_path):
    print("Loading data for Weighted Prediction...")
    item_stats = pd.read_csv(stats_path)
    centered_df = pd.read_csv(centered_path, usecols=['userId', 'movieId', 'rating_diff'])
    peers_df = pd.read_csv(peers_path)
    
    all_users = centered_df['userId'].unique()
    targets = peers_df['Target_Item'].unique()
    
    for target in targets:
        print(f"\n--- Predicting for Target {target} ---")
        
        # 1. Get Top 5 Peers and Weights
        target_peers = peers_df[peers_df['Target_Item'] == target].sort_values('Rank').head(5)
        peer_ids = target_peers['Peer_Item'].tolist()
        weights = dict(zip(target_peers['Peer_Item'], target_peers['Latent_Similarity']))
        
        print(f"Using Peers: {peer_ids}")
        
        # 2. Get User Ratings for these Peers (Efficiently)
        subset_df = centered_df[centered_df['movieId'].isin(peer_ids)].copy()
        
        # 3. Pivot to User x Peer
        user_peer_matrix = subset_df.pivot(index='userId', columns='movieId', values='rating_diff').reindex(all_users).fillna(0.0)
        
        # 4. Calculate Prediction
        # Formula: Pred = Mean + (Sum(W * (R - Mu)) / Sum(|W|))
        # Since R is already 'rating_diff' (R - Mu), we use Sum(W * rating_diff) / Sum(|W|)
        
        predictions = []
        
        # Vectorized implementation for speed
        numerator = pd.Series(0.0, index=user_peer_matrix.index)
        denominator = 0.0
        
        for pid in peer_ids:
            w = weights[pid]
            if pid in user_peer_matrix.columns:
                numerator += user_peer_matrix[pid] * w
                denominator += abs(w)
                
        if denominator == 0:
            centered_pred = 0.0
        else:
            centered_pred = numerator / denominator
            
        # Add Target Mean
        target_mean = item_stats[item_stats['movieId'] == target]['mean_rating'].values[0]
        final_pred = centered_pred + target_mean
        
        # Create Result DF
        pred_df = pd.DataFrame({
            'userId': user_peer_matrix.index,
            'movieId': target,
            'predicted_rating_centered': centered_pred,
            'predicted_rating_final': final_pred
        })
        
        out_name = f'3.2.9_predictions_target_{int(target)}.csv'
        print(f"Saving predictions to {out_name}...")
        utils.save_csv(pred_df, out_name)
        
    print("\nTask 9 Complete.")
else:
    print("Required files missing.")

Loading data for Weighted Prediction...

--- Predicting for Target 1556.0 ---
Using Peers: [2421.0, 1722.0, 256.0, 1831.0, 1370.0]
Saving predictions to 3.2.9_predictions_target_1556.csv...
    Saved CSV: tables/3.2.9_predictions_target_1556.csv

--- Predicting for Target 1499.0 ---
Using Peers: [2722.0, 1347.0, 1591.0, 3826.0, 1438.0]
Saving predictions to 3.2.9_predictions_target_1499.csv...
    Saved CSV: tables/3.2.9_predictions_target_1499.csv

Task 9 Complete.


## Task 10: Determine Reduced Dimensional Space (Top 10 Peers)
Create a reduced dimensional space for each user by selecting their centered ratings for the Top 10 peer items identified in Task 7.
Missing values are filled with 0 (since ratings are centered).

In [13]:
# Task 10: Reduced Dimensional Space (Top 10 Peers)
import pandas as pd
import os
import utils

peers_path = os.path.join('..', 'results', 'tables', '3.2.7_top_peers.csv')
centered_path = os.path.join('..', 'results', 'tables', 'task3.2.4_centered_ratings.csv')

if os.path.exists(peers_path) and os.path.exists(centered_path):
    print("Loading Top Peers and Centered Ratings (Task 10)...")
    peers_df = pd.read_csv(peers_path)
    centered_df = pd.read_csv(centered_path, usecols=['userId', 'movieId', 'rating_diff'])
    all_users = centered_df['userId'].unique()
    targets = peers_df['Target_Item'].unique()
    
    for target in targets:
        print(f"\n--- Processing Target {target} (Top 10) ---")
        
        # Top 10 Peers
        target_peers = peers_df[peers_df['Target_Item'] == target].sort_values('Rank')
        top_10_peers = target_peers.head(10)['Peer_Item'].tolist()
        print(f"Top 10 Peers: {top_10_peers}")
        
        subset_df = centered_df[centered_df['movieId'].isin(top_10_peers)]
        
        print("Creating Reduced Space Matrix...")
        reduced_matrix = subset_df.pivot(index='userId', columns='movieId', values='rating_diff')
        reduced_matrix = reduced_matrix.fillna(0.0)
        reduced_matrix = reduced_matrix.reindex(all_users, fill_value=0.0)
        
        print(f"Reduced Matrix Shape: {reduced_matrix.shape}")
        
        out_filename = f'3.2.10_reduced_space_target_{int(target)}_top10.csv'
        print(f"Saving to {out_filename}...")
        utils.save_csv(reduced_matrix.reset_index(), out_filename)
        
    print("\nTask 10 Complete.")
else:
    print("Required files missing.")

Loading Top Peers and Centered Ratings (Task 10)...

--- Processing Target 1556.0 (Top 10) ---
Top 10 Peers: [2421.0, 1722.0, 256.0, 1831.0, 1370.0, 2431.0, 2407.0, 2134.0, 1591.0, 79.0]
Creating Reduced Space Matrix...
Reduced Matrix Shape: (96345, 10)
Saving to 3.2.10_reduced_space_target_1556_top10.csv...
    Saved CSV: tables/3.2.10_reduced_space_target_1556_top10.csv

--- Processing Target 1499.0 (Top 10) ---
Top 10 Peers: [2722.0, 1347.0, 1591.0, 3826.0, 1438.0, 2949.0, 1042.0, 2067.0, 799.0, 379.0]
Creating Reduced Space Matrix...
Reduced Matrix Shape: (96345, 10)
Saving to 3.2.10_reduced_space_target_1499_top10.csv...
    Saved CSV: tables/3.2.10_reduced_space_target_1499_top10.csv

Task 10 Complete.


# Task 11 

In [14]:
# Task 11: Prediction (Top 10 Weighted Average)
import pandas as pd
import os
import utils

stats_path = os.path.join('..', 'results', 'tables', 'task3.2.3_final_item_stats.csv')
centered_path = os.path.join('..', 'results', 'tables', 'task3.2.4_centered_ratings.csv')
peers_path = os.path.join('..', 'results', 'tables', '3.2.7_top_peers.csv')

if os.path.exists(stats_path) and os.path.exists(centered_path) and os.path.exists(peers_path):
    print("Loading data for Weighted Prediction (Top 10)...")
    item_stats = pd.read_csv(stats_path)
    centered_df = pd.read_csv(centered_path, usecols=['userId', 'movieId', 'rating_diff'])
    peers_df = pd.read_csv(peers_path)
    
    all_users = centered_df['userId'].unique()
    targets = peers_df['Target_Item'].unique()
    
    for target in targets:
        print(f"\n--- Modelling for Target {target} (Top 10) ---")
        
        target_peers = peers_df[peers_df['Target_Item'] == target].sort_values('Rank').head(10)
        peer_ids = target_peers['Peer_Item'].tolist()
        weights = dict(zip(target_peers['Peer_Item'], target_peers['Latent_Similarity']))
        
        subset_df = centered_df[centered_df['movieId'].isin(peer_ids)].copy()
        user_peer_matrix = subset_df.pivot(index='userId', columns='movieId', values='rating_diff').reindex(all_users).fillna(0.0)
        
        numerator = pd.Series(0.0, index=user_peer_matrix.index)
        denominator = 0.0
        
        for pid in peer_ids:
            w = weights[pid]
            if pid in user_peer_matrix.columns:
                numerator += user_peer_matrix[pid] * w
                denominator += abs(w)
                
        if denominator == 0:
            centered_pred = 0.0
        else:
            centered_pred = numerator / denominator
            
        target_mean = item_stats[item_stats['movieId'] == target]['mean_rating'].values[0]
        final_pred = centered_pred + target_mean
        
        pred_df = pd.DataFrame({
            'userId': user_peer_matrix.index,
            'movieId': target,
            'predicted_rating_centered': centered_pred,
            'predicted_rating_final': final_pred
        })
        
        out_name = f'3.2.11_predictions_target_{int(target)}_top10.csv'
        print(f"Saving predictions to {out_name}...")
        utils.save_csv(pred_df, out_name)
        
    print("\nTask 11 Complete.")
else:
    print("Required files missing.")

Loading data for Weighted Prediction (Top 10)...

--- Modelling for Target 1556.0 (Top 10) ---
Saving predictions to 3.2.11_predictions_target_1556_top10.csv...
    Saved CSV: tables/3.2.11_predictions_target_1556_top10.csv

--- Modelling for Target 1499.0 (Top 10) ---
Saving predictions to 3.2.11_predictions_target_1499_top10.csv...
    Saved CSV: tables/3.2.11_predictions_target_1499_top10.csv

Task 11 Complete.


# Task 12

In [15]:
# Task 12: Compare Results
results_dir = os.path.join('..', 'results', 'tables')
targets = [1499, 1556]

for target_id in targets:
    file_top5 = os.path.join(results_dir, f'3.2.9_predictions_target_{target_id}.csv')
    file_top10 = os.path.join(results_dir, f'3.2.11_predictions_target_{target_id}_top10.csv')
    
    if os.path.exists(file_top5) and os.path.exists(file_top10):
        print(f"\n--- Comparison for Target {target_id} ---")
        
        df5 = pd.read_csv(file_top5)
        df10 = pd.read_csv(file_top10)
        
        # Rename columns for clear merging
        df5 = df5[['userId', 'predicted_rating_final']].rename(columns={'predicted_rating_final': 'pred_top5'})
        df10 = df10[['userId', 'predicted_rating_final']].rename(columns={'predicted_rating_final': 'pred_top10'})
        
        # Merge on userId to align predictions
        merged = df5.merge(df10, on='userId', how='inner')
        
        # Calculate Difference
        merged['diff'] = merged['pred_top10'] - merged['pred_top5']
        merged['abs_diff'] = merged['diff'].abs()
        
        # Statistics
        print(f"Total Predictions Compared: {len(merged)}")
        print(f"Mean Difference (Top10 - Top5): {merged['diff'].mean():.4f}")
        print(f"Mean Absolute Difference: {merged['abs_diff'].mean():.4f}")
        print(f"Max Difference: {merged['abs_diff'].max():.4f}")
        
        print("\nSample Comparison (First 5 users):")
        print(merged.head(5).to_string(index=False))
        
        # Explanation logic (dynamic print)
        if merged['abs_diff'].mean() < 0.1:
            print("\nConclusion: The predictions are very similar. Adding 5 more peers didn't drastically change the outcome.")
        else:
            print("\nConclusion: There is a noticeable difference. The additional 5 peers influenced the regression model.")
            
    else:
        print(f"Missing files for target {target_id}")


--- Comparison for Target 1499 ---
Total Predictions Compared: 96345
Mean Difference (Top10 - Top5): 0.0000
Mean Absolute Difference: 0.0036
Max Difference: 0.4813

Sample Comparison (First 5 users):
 userId  pred_top5  pred_top10  diff  abs_diff
      1   2.059603    2.059603   0.0       0.0
      2   2.059603    2.059603   0.0       0.0
      3   2.059603    2.059603   0.0       0.0
      4   2.059603    2.059603   0.0       0.0
      5   2.059603    2.059603   0.0       0.0

Conclusion: The predictions are very similar. Adding 5 more peers didn't drastically change the outcome.

--- Comparison for Target 1556 ---
Total Predictions Compared: 96345
Mean Difference (Top10 - Top5): 0.0000
Mean Absolute Difference: 0.0046
Max Difference: 0.3692

Sample Comparison (First 5 users):
 userId  pred_top5  pred_top10     diff  abs_diff
      1   1.833879    1.872520 0.038641  0.038641
      2   1.919431    1.919431 0.000000  0.000000
      3   1.919431    1.919431 0.000000  0.000000
      4   

In [16]:
# --- DYNAMIC BENCHMARKING: PCA Mean Filling (Part 1) ---\n
method_name = "PCA Mean Filling (Part 1)"
import time
import pandas as pd
import numpy as np
import os
from scipy.linalg import eigh
# Ensure utils is loaded if locals missing (fallback)
if 'load_data' not in locals():
    try:
        from utils import load_data, calculate_mle_covariance
    except ImportError:
        print("Warning: utils module not found. Benchmarking might fail.")

print(f"\n--- Benchmarking {method_name} ---")

# 1. Setup Results Path
results_table_dir = os.path.join(os.getcwd(), '../results/tables')
if not os.path.exists(results_table_dir):
   os.makedirs(results_table_dir)

# 2. Data Preparation (Robust Loading)
R_bench = None

# Attempt 1: Check for existing pivoted variables
if 'user_item_matrix' in locals():
    R_bench = user_item_matrix
elif 'R_df' in locals():
    R_bench = R_df

# Attempt 2: Check for raw ratings and pivot
if R_bench is None:
    if 'ratings_df' in locals():
        print("Pivoting ratings_df for benchmark...")
        R_bench = ratings_df.pivot(index='userId', columns='movieId', values='rating')
    elif 'df' in locals() and 'rating' in df.columns:
        print("Pivoting df for benchmark...")
        R_bench = df.pivot(index='userId', columns='movieId', values='rating')

# Attempt 3: Load from Disk
if R_bench is None and 'load_data' in locals():
    print("Loading data from disk for benchmark...")
    _df = load_data()
    if _df is not None:
        R_bench = _df.pivot(index='userId', columns='movieId', values='rating')

if R_bench is None:
    print("Error: Could not obtain User-Item Matrix. Skipping Benchmark.")
else:
    # Ensure float
    R_bench = R_bench.astype(float)
    
    # 3. Decomposition & Prediction
    t0 = time.time()
    
    if 'Mean Filling' in method_name:
        # Mean Filling Logic
        item_means_bench = R_bench.mean()
        R_filled_bench = R_bench.fillna(item_means_bench)
        R_centered_bench = R_filled_bench - item_means_bench
        X = R_centered_bench.values
        # Cov = X.T @ X / (N-1)
        cov_matrix_bench = np.dot(X.T, X) / (X.shape[0] - 1)
        
    else:
        # MLE Logic
        if 'calculate_mle_covariance' in locals():
             cov_matrix_bench, _, _ = calculate_mle_covariance(R_bench)
             if isinstance(cov_matrix_bench, pd.DataFrame): cov_matrix_bench = cov_matrix_bench.values
        else:
             print("MLE Util not found. Skipping.")
             cov_matrix_bench = None

    if cov_matrix_bench is not None:
        # Eigh
        evals_b, evecs_b = eigh(cov_matrix_bench)
        idx_b = np.argsort(evals_b)[::-1]
        evecs_b = evecs_b[:, idx_b]
        
        t1 = time.time()
        time_decomp = t1 - t0
        
        # 4. Prediction Timing
        t2 = time.time()
        k_limit = 50
        V_k = evecs_b[:, :k_limit]
        
        if 'Mean Filling' in method_name:
             # U = X V
             U_b = np.dot(X, V_k)
             # X_hat = U V.T + Mean
             X_hat_b = np.dot(U_b, V_k.T) + item_means_bench.values
        else:
             means_mle = R_bench.mean()
             X_mle = R_bench.fillna(means_mle).values - means_mle.values
             U_b = np.dot(X_mle, V_k)
             X_hat_b = np.dot(U_b, V_k.T) + means_mle.values
             
        t3 = time.time()
        time_pred = t3 - t2
        
        # 5. Metrics
        mask_b = ~np.isnan(R_bench.values)
        diff_b = R_bench.values[mask_b] - X_hat_b[mask_b]
        rmse_val = np.sqrt(np.mean(diff_b**2))
        mae_val = np.mean(np.abs(diff_b))
        mem_mb = (cov_matrix_bench.nbytes + evecs_b.nbytes) / 1024 / 1024
        
        print(f"Results: Time Decomp {time_decomp:.4f}s, RMSE {rmse_val:.4f}")
        
        # 6. Save
        bench_file = os.path.join(results_table_dir, 'pca_benchmarks.csv')
        new_data = {
            'Method': method_name,
            'k': k_limit,
            'RMSE': rmse_val,
            'MAE': mae_val,
            'Time_Decomp(s)': time_decomp,
            'Time_Pred(s)': time_pred,
            'Memory(MB)': mem_mb
        }
        
        if os.path.exists(bench_file):
            df_bench = pd.read_csv(bench_file)
            df_bench = df_bench[df_bench['Method'] != method_name]
            df_bench = pd.concat([df_bench, pd.DataFrame([new_data])], ignore_index=True)
        else:
            df_bench = pd.DataFrame([new_data])
            
        df_bench.to_csv(bench_file, index=False)
        print(f"Benchmark saved to {bench_file}")



--- Benchmarking PCA Mean Filling (Part 1) ---
Pivoting ratings_df for benchmark...
Results: Time Decomp 3.1502s, RMSE 0.8281
Benchmark saved to d:\University\semester 9\IRS\AIE425_FinalProject_Group17\SECTION1_DimensionalityReduction\code\../results/tables\pca_benchmarks.csv
